In [1]:
import numpy as np
np.set_printoptions(precision=3) # only 3 decimals in print
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import animation
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf')
from scipy.integrate import solve_ivp, quad
from scipy.optimize import fsolve
from scipy.spatial.distance import pdist, squareform

plt.rc("axes", labelsize = 11)
plt.rc("xtick", labelsize = 10, top = True, direction="in")
plt.rc("ytick", labelsize = 10, right = True, direction="in")
plt.rc("axes", titlesize = 13)
plt.rc("legend", fontsize = 10, loc = "best")
plt.rc('animation', html='jshtml')

In [2]:
class PeriodicLennardJones:
    def __init__(self,eps=1):
        self.eps = eps
        
    def _V(self,r):
        return self.eps * (r**(-12) - 2 * r**(-6))

    def _dV_dr(self, r):
        return self.eps * (-12 * r**(-13) + 12 * r**(-7) )
    
    def _pairwise_distance_matrix(self, pos, box):
        diff = pos[np.newaxis, :, :] - pos[:, np.newaxis, :]
        for dim in range(2):  # Loop over x and y dimensions and get smallest component
            diff[..., dim] -= np.rint(diff[..., dim] / box[dim, dim]) * box[dim, dim]
        return diff
    
    def energy(self, pos, box):
        diff = self._pairwise_distance_matrix(pos, box)
        r = np.sqrt(np.sum(diff**2, axis=-1))
        return np.sum(self._V(squareform(r)))
    
    def forces(self, pos, box):
        diff = self._pairwise_distance_matrix(pos, box)
        r = np.sqrt(np.sum(diff**2, axis=-1))
        np.fill_diagonal(r, 1000)
        force_magnitude = self._dV_dr(r)
        forces = np.sum(force_magnitude[..., np.newaxis] * diff / \
                        r[..., np.newaxis], axis=1)
        return forces

In [3]:
class PeriodicSystem:
    def __init__(self, calc, N=None, pos=None, static=None, box=None,
                descriptor_method=None, kT=0.15):
        self.calc = calc
        self.box = box
        assert (N is not None and pos is None) or \
               (N is None and pos is not None), 'You must specify either N or pos'
        if pos is not None:
            self.set_positions(np.array(pos)*1.)
            self.N = len(pos)
        else:
            self.N = N
            pos = np.random.rand(N,2)
            pos[:,0] *= self.box[0,0]
            pos[:,1] *= self.box[1,1]
            self.set_positions(pos)
        if static is not None:
            assert len(static) == self.N, 'static must be N long'
            self.static = static
        else:
            self.static = [False for _ in range(self.N)]
        self.filter = np.array([self.static,self.static]).T
        self.indices_dynamic_atoms = \
                            [i for i,static in enumerate(self.static) if not static]
        self.plot_artists = {}
        self.descriptor_method = descriptor_method
        self.kT = kT
        self.velocities = np.zeros(self.pos.shape)
        
    def copy(self):
        return PeriodicSystem(self.calc, pos=self.pos, static=self.static,
                              box=self.box)

    def stress(self, delta=1e-4):
        return self.calc.stress(self.pos, self.box, delta=delta)
    
    def pressure(self, delta=1e-4):
        return self.calc.pressure(self.pos, self.box, delta=delta)
    
    @property
    def kinetic_energy(self):
        return 0.5 * np.sum(self.velocities**2)

    def set_velocities(self,velocities):
        self.velocities = np.where(self.filter,0,velocities)

    def get_velocities(self):
        return self.velocities.copy()
    
    def energy_title(self):
        return f'Ek={self.kinetic_energy:.1f} ' +\
               f'Ep={self.potential_energy:.1f} ' + \
               f'E={self.potential_energy + self.kinetic_energy:.1f}'
    
    @property
    def potential_energy(self):
        return self.calc.energy(self.pos,self.box)
        
    def forces(self):
        forces = self.calc.forces(self.pos,self.box)
        return np.where(self.filter,0,forces)
    
    def set_positions(self,pos):
        pos[:,0] = pos[:,0] % self.box[0,0]
        pos[:,1] = pos[:,1] % self.box[1,1]
        self.pos = pos
        
    def get_scaled_positions(self):
        return self.pos @ np.linalg.inv(self.box).T
        
    def get_positions(self):
        return self.pos.copy()
        
    def draw_periodic(self,ax,size=200):
        ax.scatter(self.pos[:,0],self.pos[:,1],s=size)
        for i, (x,y) in enumerate(self.pos):
            ax.text(x,y,str(i),ha='center',va='center',color='w')
        deltas = [(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]
        for dx,dy in deltas:
            ax.scatter(self.pos[:,0]+dx*self.box[0,0],self.pos[:,1]+dy*self.box[1,1],
                       s=size,facecolor='w',edgecolor='C0')
    def draw_cell(self,ax):        
        ax.plot(np.array([0,self.box[0,0],self.box[0,0],0,0]),
                np.array([0,0,self.box[1,1],self.box[1,1],0]))
        
    @property
    def descriptor(self):
        return self.descriptor_method.descriptor(self.pos)

    def energy_title(self):
        return f'Ep={self.potential_energy:.1f}'
    
    def draw(self,ax,size=100,alpha=1,force_draw=False,edge=False,color='C0',
             energy_title=True):
        if self.plot_artists.get(ax,None) is None or force_draw or edge:
            colors = ['C1' if s else color for s in self.static]
            facecolors = [to_rgba(c,alpha) for c in colors]
            if edge:
                edgecolors = (0,0,0,1)
                self.plot_artists[ax] = ax.scatter(self.pos[:,0],self.pos[:,1],
                                               s=size,facecolors=facecolors,
                                                  edgecolors=edgecolors,
                                                  linewidth=2)
            else:
                edgecolors = (0,0,0,0.5)
                self.plot_artists[ax] = ax.scatter(self.pos[:,0],self.pos[:,1],
                                                   s=size,facecolors=facecolors,
                                                  edgecolors=edgecolors,)
        else:
            self.plot_artists[ax].set_offsets(self.pos)
        if energy_title:
            ax.set_title(self.energy_title())

    def draw_descriptor(self,ax):
        self.descriptor_method.draw(self.pos,ax)